## IBM Data Science Specialization - Final Project

This is the notebook for Darvesh Gorhe's capstone project for the IBM Data Specialization on Coursera

In [ ]:
import pandas as pd
import numpy as np
import requests
import random

from IPython.display import Image
from IPython.core.display import HTML

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Libraries imported.')

print("Hello Capstone Project Course!")

In [1]:
{
    "tags": [
        "hide_input",
    ]
}

CLIENT_ID = 'KGXJMPLRR52DZFGT3KTZPE3ZNVPYSAINZ1FWVEZLEHYOSMZZ' # Foursquare ID
CLIENT_SECRET = 'RJWEQT0O1URM0LDVVN4YJ4E41LK0QOBFQKAYLF32W5LSIJMI' # Foursquare Secret
VERSION = '20200326'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## A. Introduction

### A.1 Problem Outline
For this project, I wanted to compare the coffee scene in four US cities: San Francisco, Los Angeles, Seattle, and New York. The purpose of this is to understand which cities high competition among coffee shops and where would be a good place to invest in more coffee shops. If I was a big coffee chain and I wanted to determine where to open more coffee shops, this data would be useful in determining where to open new storefronts among these cities. Of course this could be extended to more cities but for this project, we'll limit it to these 4 cities. 

### A.2 Data Used
I will be using the following data from each city for my analysis:
* Absolute number of coffee shops 
* Average number of hours the shops are open
* Median number of hours the shops are open
* Standard deviation in coffee shop hours 

Furthermore I will also be using k-means clustering to create groups of 

17
